In [1]:
import pandas as pd
import numpy as np
import pgeocode

In [2]:
COLS_TO_DROP = [
    'Date received',
    'Consumer complaint narrative',
    'Company public response',
    'Tags',
    'Date sent to company',
    'Timely response?',
    'Consumer disputed?',
    'Complaint ID'
]

In [3]:
STATE_CAPITAL_ZIPS = {
    'AL': '36104', 'AK': '99801', 'AZ': '85007', 'AR': '72201', 'CA': '95814',
    'CO': '80203', 'CT': '06106', 'DE': '19901', 'FL': '32301', 'GA': '30334',
    'HI': '96813', 'ID': '83702', 'IL': '62701', 'IN': '46225', 'IA': '50319',
    'KS': '66612', 'KY': '40601', 'LA': '70802', 'ME': '04330', 'MD': '21401',
    'MA': '02133', 'MI': '48933', 'MN': '55155', 'MS': '39201', 'MO': '65101',
    'MT': '59620', 'NE': '68508', 'NV': '89701', 'NH': '03301', 'NJ': '08608',
    'NM': '87501', 'NY': '12210', 'NC': '27601', 'ND': '58501', 'OH': '43215',
    'OK': '73102', 'OR': '97301', 'PA': '17120', 'RI': '02903', 'SC': '29201',
    'SD': '57501', 'TN': '37219', 'TX': '78701', 'UT': '84114', 'VT': '05602',
    'VA': '23219', 'WA': '98504', 'WV': '25301', 'WI': '53703', 'WY': '82001',
    'AS': '96799', 'DC': '20001', 'FM': '96941', 'GU': '96910', 'MH': '96960',
    'MP': '96950', 'PR': '00901', 'PW': '96940', 'VI': '00802'
}

In [4]:
def clean_zip_code_part4(zip_code):
    if pd.isna(zip_code):
        return None
    zip_str = str(zip_code).split('.')[0]
    zip_digits = ''.join(filter(str.isdigit, zip_str))
    if len(zip_digits) < 5:
        return zip_digits.zfill(5)
    elif len(zip_digits) > 5:
        return zip_digits[:5]
    return zip_digits

In [5]:
nomi = pgeocode.Nominatim('us')

def get_state_from_zip(zip_code):
    if pd.isna(zip_code) or not isinstance(zip_code, str) or len(zip_code) != 5:
        return None
    try:
        query = nomi.query_postal_code(zip_code)
        if pd.notna(query['state_code']):
            return query['state_code']
        else:
            return None
    except Exception:
        return None

In [6]:
input_filename = "complaints-2025-05-01_22_48.csv"
try:
    temp_df_cols = pd.read_csv(input_filename, nrows=0).columns
    zip_col_name = None
    possible_zip_names = ['ZIP code', 'zip code', 'Zip code', 'Postal code']
    for name in possible_zip_names:
        if name in temp_df_cols:
            zip_col_name = name
            break
    if not zip_col_name:
        raise ValueError("Could not find a suitable ZIP code column name.")
    df = pd.read_csv(input_filename, dtype={zip_col_name: str})
    print(f"Success and shape: {df.shape}")
    print(f"Initial dtype of '{zip_col_name}': {df[zip_col_name].dtype}")
except FileNotFoundError:
    print(f"Error: {input_filename}")
    exit()
except ValueError as ve:
     print(f"Error: {ve}")
     exit()
except Exception as e:
    print(f"Error loading data: {e}")
    exit()

Success and shape: (139894, 18)
Initial dtype of 'ZIP code': object


C:\Users\ncpr0\AppData\Local\Temp\ipykernel_12380\1495614252.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filename, dtype={zip_col_name: str})


In [7]:
initial_rows = len(df)
cols_to_drop_existing = [col for col in COLS_TO_DROP if col in df.columns]
df.drop(columns=cols_to_drop_existing, inplace=True)
print(f"Dropped columns: {cols_to_drop_existing}")
print(f"Shape after dropping columns: {df.shape}")

Dropped columns: ['Date received', 'Consumer complaint narrative', 'Company public response', 'Tags', 'Date sent to company', 'Timely response?', 'Consumer disputed?', 'Complaint ID']
Shape after dropping columns: (139894, 10)


In [8]:
df['zip_code_clean'] = df[zip_col_name].apply(clean_zip_code_part4)
print(f"Cleaned ZIP code column created dtype: {df['zip_code_clean'].dtype}")

Cleaned ZIP code column created dtype: object


In [9]:
if zip_col_name in df.columns:
    df.drop(columns=[zip_col_name], inplace=True)
    print(f"Original column '{zip_col_name}' dropped.")

Original column 'ZIP code' dropped.


In [10]:
state_col_name = None
possible_state_names = ['State', 'state']
for name in possible_state_names:
    if name in df.columns:
        state_col_name = name
        break

In [11]:
if not state_col_name:
    print("Could not find a state column")
else:
    print(f"Handling missing values in '{state_col_name}' and zip_code_clean")
    print(f"Initial missing States: {df[state_col_name].isnull().sum()}")
    print(f"Initial missing Cleaned ZIPs: {df['zip_code_clean'].isnull().sum()}")

    missing_zip_mask = df['zip_code_clean'].isnull() & df[state_col_name].notnull()
    num_imputed_zips = missing_zip_mask.sum()
    if num_imputed_zips > 0:
        print(f"Attempting to impute {num_imputed_zips} missing ZIP codes using state capitals")
        df.loc[missing_zip_mask, 'zip_code_clean'] = df.loc[missing_zip_mask, state_col_name].map(STATE_CAPITAL_ZIPS)
        print(f"Imputed ZIPs based on state. Missing ZIPs remaining: {df['zip_code_clean'].isnull().sum()}")
    else:
        print("No missing ZIPs found where state was present.")

    missing_state_mask = df[state_col_name].isnull() & df['zip_code_clean'].notnull()
    num_imputed_states = missing_state_mask.sum()
    if num_imputed_states > 0:
        print(f"Attempting to impute {num_imputed_states} missing States using ZIP codes")
        imputed_states = df.loc[missing_state_mask, 'zip_code_clean'].apply(get_state_from_zip)
        df.loc[missing_state_mask, state_col_name] = imputed_states
        print(f"Imputed States based on ZIP. Missing States remaining: {df[state_col_name].isnull().sum()}")
    else:
        print("No missing States found where ZIP was present.")
        
    rows_before_drop = len(df)
    df = df[df[state_col_name].notnull() | df['zip_code_clean'].notnull()]
    rows_after_drop = len(df)
    print(f"Dropped {rows_before_drop - rows_after_drop} rows where both state and cleaned ZIP were missing.")
    print(f"Shape after handling missing State/ZIP: {df.shape}")

Handling missing values in 'State' and zip_code_clean
Initial missing States: 2574
Initial missing Cleaned ZIPs: 1633
Attempting to impute 11 missing ZIP codes using state capitals
Imputed ZIPs based on state. Missing ZIPs remaining: 1622
Attempting to impute 952 missing States using ZIP codes
Imputed States based on ZIP. Missing States remaining: 2532
Dropped 1622 rows where both state and cleaned ZIP were missing.
Shape after handling missing State/ZIP: (138272, 10)


In [12]:
final_col_names = [
    'Product',
    'Sub-product',
    'Issue',
    'Sub-issue',
    'Company response to consumer',
    'Company',
    state_col_name, 
    'zip_code_clean' 
]

In [13]:
final_cols_existing = [col for col in final_col_names if col in df.columns]

df_cleaned = df[final_cols_existing].copy() 
df_cleaned['zip_code_clean'] = df_cleaned['zip_code_clean'].astype(str)
if state_col_name in df_cleaned.columns:
    df_cleaned[state_col_name] = df_cleaned[state_col_name].astype(str)

In [14]:
print(f"Final columns selected: {final_cols_existing}")

print("\n--- Final Cleaned Data Overview ---")
print(f"Final Shape: {df_cleaned.shape}")
print(f"Rows remaining: {len(df_cleaned)} (out of {initial_rows})")
print("\nData Types:")
print(df_cleaned.dtypes)
print("\nMissing values per column:")
print(df_cleaned.isnull().sum()) 
print("\nFirst 5 rows:")
print(df_cleaned.head())

# output_filename = "part_4_data_cleaning.csv"
# try:
#     df_cleaned.to_csv(output_filename, index=False)
#     print(f"\nCleaned data successfully saved to '{output_filename}'")
# except Exception as e:
#     print(f"\nError saving cleaned data: {e}")

# print("\nData cleaning script finished.")



Final columns selected: ['Product', 'Sub-product', 'Issue', 'Sub-issue', 'Company response to consumer', 'Company', 'State', 'zip_code_clean']

--- Final Cleaned Data Overview ---
Final Shape: (138272, 8)
Rows remaining: 138272 (out of 139894)

Data Types:
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Company response to consumer    object
Company                         object
State                           object
zip_code_clean                  object
dtype: object

Missing values per column:
Product                             0
Sub-product                     10521
Issue                               0
Sub-issue                       52613
Company response to consumer        0
Company                             0
State                               0
zip_code_clean                      0
dtype: int64

First 5 rows:
                                             Product  \
0